# Part C

## Blanco 1 cluster

Modelling over Blanco 1 cluster's mass distribution

In [1]:
%pip install numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:

import numpy as np
import matplotlib.pyplot as plt
from subprocess import run
import os
import mesa_reader
import multiprocessing as mp

In [3]:

def salpeter_imf(mass_min, mass_max, num_stars):
    """Generate masses following Salpeter IMF: dN/dM ∝ M^(-2.35)"""
    alpha = -2.35
    u = np.random.random(num_stars)
    alpha1 = alpha + 1
    m1 = mass_min**alpha1
    m2 = mass_max**alpha1
    masses = (m1 + (m2 - m1)*u)**(1/alpha1)
    return masses

In [4]:
def inlist_pgstar(mass):
    return f"""
&pgstar
  ! see star/defaults/pgstar.defaults

  ! MESA uses PGPLOT for live plotting and gives the user a tremendous
  ! amount of control of the presentation of the information.

  ! show HR diagram
  ! this plots the history of L,Teff over many timesteps

    TRho_Profile_win_width = 6
    TRho_Profile_win_aspect_ratio = 1.0
    TRho_Profile_win_flag = .true.
    TRho_Profile_file_flag = .true.
    TRho_Profile_file_dir = 'pgstar_out_{mass}'
    TRho_Profile_file_prefix = 'trho_pgstar_{mass}'
    TRho_Profile_file_interval = 50
    TRho_Profile_win_width = 9
    TRho_Profile_win_aspect_ratio = 1.0
    TRho_Profile_title = 'TRho'


  ! set static plot bounds
    HR_logT_min = 3.5
    HR_logT_max = 4.6
    HR_logL_min = 2.0
    HR_logL_max = 6.0

  ! set window size (aspect_ratio = height/width)
    HR_win_width = 6
    HR_win_aspect_ratio = 1.0
    HR_win_flag = .true.
    HR_file_flag = .true.
    HR_file_dir = 'pgstar_out_{mass}'
    HR_file_prefix = 'hr_pgstar_{mass}'
    HR_file_interval = 50
    HR_win_width = 9
    HR_win_aspect_ratio = 1.0
    HR_title = 'HR'



  ! show temperature/density profile
  ! this plots the internal structure at single timestep
    TRho_Profile_win_flag = .true.

  ! add legend explaining colors
    show_TRho_Profile_legend = .true.

  ! display numerical info about the star
    show_TRho_Profile_text_info = .true.

  ! set window size (aspect_ratio = height/width)
    TRho_Profile_win_width = 8
    TRho_Profile_win_aspect_ratio = 0.75

/ ! end of pgstar namelist
"""


def inlist_project(mass):
    return f"""
&star_job

    create_pre_main_sequence_model = .true.

    save_model_when_terminate = .false.
    save_model_filename = '15M_at_TAMS.mod'

    pgstar_flag = .true.

/ 


&kap
  use_Type2_opacities = .true.
  Zbase = 0.02

/

&controls
    initial_mass = {mass} ! in Msun units
    initial_z = 0.02

    Lnuc_div_L_zams_limit = 0.99d0
    stop_near_zams = .true.

    xa_central_lower_limit_species(1) = 'h1'
    xa_central_lower_limit(1) = 1d-3

     energy_eqn_option = 'dedt'
     use_gold_tolerances = .true.


/
"""

In [5]:
mass_l_t = dict()
results = []

def run_mesa_model(masses):
    for mass in masses:
        """Run MESA for a single mass"""
        print(f"FOR MASS: {mass}")
        
        with open('star/inlist_pgstar', 'w') as f:
            f.write(inlist_pgstar(mass))
        
        with open('star/inlist_project', 'w') as f:
            f.write(inlist_project(mass))
        
        # run(['cd star'], check=True)
        run(['pwd'], check=True)
        run(['ls'], check=True)
        os.chdir("./star")
        run(['pwd'], check=True)
        run(['./mk'], check=True)
        run(['./rn'], check=True)
        os.chdir("../")
        
        if os.path.exists('star/LOGS/history.data'):
            results.append({'mass': mass, 'success': True})
        else:
            results.append({'mass': mass, 'success': False})

        mr = mesa_reader.MesaData(file_name='./star/LOGS/history.data')
        mass_l_t[mass] = (mr.log_L, mr.log_Teff)



  

In [6]:
def plot_hr_diagram(masses):
    """Plot HR diagram for all stars in the same plot"""
    plt.figure()
    for mass in masses:
        plt.plot(mass_l_t[mass][1], mass_l_t[mass][0], label=f'{mass:.2f} Msun')
    plt.gca().invert_xaxis()
    plt.xlabel('log(Teff)')
    plt.ylabel('log(L)')
    plt.legend()
    plt.show()

In [ ]:
masses = salpeter_imf(0.2, 8, 644)

run_mesa_model(masses)

plot_hr_diagram(masses)

FOR MASS: 2.90167221109542
/root/astrolab
mass_dist.py
mass_distribution.png
partA.ipynb
partB.ipynb
star
/root/astrolab/star
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/root/mesa-24.08.1/lib -lstar -lgyre_mesa -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lforum -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


/root/mesasdk/bin/../lib/gcc/x86_64-pc-linux-gnu/13.3.0/../../../../x86_64-pc-linux-gnu/bin/ld: warning: atm_support.o: requires executable stack (because the .note.GNU-stack section is executable)


DATE: 2025-03-04
TIME: 14:25:55
 reading user weak rate file /root/mesa-24.08.1/data/rates_data/rate_tables/S13_r_be7_wk_li7.h5
 version_number r24.08.1
 read inlist_project


 The terminal output contains the following information

      'step' is the number of steps since the start of the run,
      'lg_dt' is log10 timestep in years,
      'age_yr' is the simulated years since the start run,
      'lg_Tcntr' is log10 center temperature (K),
      'lg_Dcntr' is log10 center density (g/cm^3),
      'lg_Tmax' is log10 max temperature (K),
      'Teff' is the surface temperature (K),
      'lg_R' is log10 surface radius (Rsun),
      'lg_L' is log10 surface luminosity (Lsun),
      'lg_LH' is log10 total PP and CNO hydrogen burning power (Lsun),
      'lg_L3a' is log10 total triple-alpha helium burning power (Lsun),
      'lg_gsurf' is log10 surface gravity,
      'lg_LNuc' is log10 nuclear power (Lsun),
      'lg_LNeu' is log10 total neutrino power (Lsun),
      'lg_Lphoto' is log10 to

PGPLOT /xw: cannot connect to X server [:0]
PGPLOT /xw: cannot connect to X server [:0]


          3   6.389859   4943.959  -7.801937  -7.801937   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978143    628      0
-4.8416E+00   6.389859   0.961946 -99.000000  -9.199801 -99.000000   0.000000   0.280000   0.009381   0.020000   0.035581      2
 3.6400E-05  -1.467737   1.654885 -23.179310 -99.000000  -7.455021   0.000000   0.003447   0.002085   0.020000  0.000E+00  max increase

          4   6.389859   4943.953  -7.801937  -7.801937   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978143    628      0
-4.7625E+00   6.389859   0.961946 -99.000000  -9.199801 -99.000000   0.000000   0.280000   0.009381   0.020000   0.035581      2
 5.3680E-05  -1.467737   1.654882 -99.000000 -99.000000  -7.455020   0.000000   0.003447   0.002085   0.020000  0.000E+00  max increase

          5   6.389859   4943.944  -7.801937  -7.801937   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978143    628      0
-4.6833E+00   6.389859   0.961946 -99.000000  -9.199801 -99.

KeyboardInterrupt: 

         52   6.389860   4943.870  -7.801925  -7.801925   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978139    628      0
-9.6176E-01   6.389860   0.961945 -99.000000  -9.199790 -99.000000   0.000000   0.280000   0.009381   0.020000   0.035581      2
 6.5518E-01  -1.467734   1.654851 -99.000000 -99.000000  -7.455009   0.000000   0.003447   0.002085   0.020000  0.000E+00  max increase

         53   6.389860   4943.870  -7.801923  -7.801923   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978138    628      0
-8.8258E-01   6.389860   0.961944 -99.000000  -9.199788 -99.000000   0.000000   0.280000   0.009381   0.020000   0.035581      2
 7.8623E-01  -1.467733   1.654850 -99.000000 -99.000000  -7.455009   0.000000   0.003447   0.002085   0.020000  0.000E+00  max increase

         54   6.389861   4943.870  -7.801920  -7.801920   2.901672   2.901672   0.700000   0.001010   0.280000  -6.978137    628      0
-8.0339E-01   6.389861   0.961944 -99.000000  -9.199785 -99.